In [1]:
import pandas as pd 
import datetime
from datetime import datetime,timedelta

from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.arima_model import ARIMA
import statsmodels.api as sm

from sklearn.preprocessing import StandardScaler
from pmdarima.arima.utils import ndiffs
from pmdarima import auto_arima


In [1]:
# Database related all the code is moved to this file
%run -i "SqlConn.py"
#importing local py file
import SqlConn 

In [5]:
#Fetch only Finance Data
try:
    df_dbFianceData = SqlConn.fetchFinanceData("")
    print("Data transfer Done")
except BaseException as err:
    print(err)


Data transfer Done


In [6]:
#Fetch Tweets and Sentiment data - Join from Tweet and Sentiment tables
try:
    df_dbSentimentData = SqlConn.fetchTweetSentimentForModelling("")
    print("Data transfer Done")
except BaseException as err:
    print(err)


Data transfer Done


In [6]:
df_dbFianceData

,financeType,date,open,high,low,close,adjustedClose,volume
0,tesla,2010-06-29,3.800000,5.000000,3.508000,4.778000,4.778000,93831500
1,tesla,2010-06-30,5.158000,6.084000,4.660000,4.766000,4.766000,85935500
2,tesla,2010-07-01,5.000000,5.184000,4.054000,4.392000,4.392000,41094000
3,tesla,2010-07-02,4.600000,4.620000,3.742000,3.840000,3.840000,25699000
4,tesla,2010-07-06,4.000000,4.000000,3.166000,3.222000,3.222000,34334500
...,...,...,...,...,...,...,...,...
6916,doge,2022-07-09,0.069224,0.070009,0.068995,0.069512,0.069512,254272147
6917,doge,2022-07-10,0.069509,0.069509,0.066533,0.067263,0.067263,299810124
6918,doge,2022-07-11,0.067268,0.067268,0.061535,0.061847,0.061847,326125900
6919,doge,2022-07-12,0.061851,0.062861,0.060087,0.060087,0.060087,298462083


In [3]:
df_dbSentimentData

,financeType,date,open,high,low,close,adjustedClose,volume
